# Description of the dataset and the task
- Data Collection
- Implications on the types of conclusions that could be made from the data
- Description of the variables, observations, and/or structure of the data
- Target task

In [1]:
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)

2.3.0


In [36]:
# Create a list with the filepaths for training and testing
batch_size = 32
img_height = 244
img_width = 244

train_path = Path('dataset/train')

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path, 
    validation_split=0.2, 
    subset="training", 
    seed=123, 
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

AttributeError: module 'tensorflow.keras.utils' has no attribute 'image_dataset_from_directory'

In [22]:
print('----- Training set -----')
print(f'Number of images: {train_df.shape[0]}\n')
print(f'Number of different labels: {len(train_df.Labels.unique())}\n')
print(f'Labels: {train_df.Labels.unique()}')

print('\n----- Test set -----')
print(f'Number of images: {test_df.shape[0]}\n')
print(f'Number of different labels: {len(test_df.Labels.unique())}\n')
print(f'Labels: {test_df.Labels.unique()}')

----- Training set -----
Number of images: 3581

Number of different labels: 36

Labels: ['tomato' 'cabbage' 'bell pepper' 'lettuce' 'chilli pepper' 'lemon'
 'jalepeno' 'beetroot' 'grapes' 'potato' 'corn' 'orange' 'eggplant'
 'carrot' 'sweetpotato' 'paprika' 'watermelon' 'cucumber' 'apple'
 'cauliflower' 'pomegranate' 'capsicum' 'pineapple' 'mango' 'pear' 'peas'
 'kiwi' 'soy beans' 'sweetcorn' 'raddish' 'onion' 'garlic' 'turnip'
 'banana' 'spinach' 'ginger']

----- Validation set -----
Number of images: 351

Number of different labels: 36

Labels: ['apple' 'eggplant' 'kiwi' 'onion' 'potato' 'banana' 'cauliflower'
 'pomegranate' 'soy beans' 'bell pepper' 'lemon' 'pineapple' 'cabbage'
 'raddish' 'corn' 'sweetpotato' 'chilli pepper' 'orange' 'grapes' 'carrot'
 'spinach' 'beetroot' 'ginger' 'mango' 'turnip' 'paprika' 'pear'
 'watermelon' 'capsicum' 'garlic' 'peas' 'tomato' 'sweetcorn' 'jalepeno'
 'cucumber' 'lettuce']

----- Test set -----
Number of images: 359

Number of different labels:

In [ ]:
# The DataFrame with the filepaths in one column and the labels in the other one
train_df.head(5)

# Exploratory data analysis
- Mean Image
- Distribution of the Size of Images
- Distribution of labels

# Data Pre-processing and Cleaning
- Change into same dimensions

In [15]:
# load and augment the images for the CNN model
pp_function = tf.keras.applications.mobilenet_v2.preprocess_input
train_img_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=pp_function)
test_img_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=pp_function)

train_images = train_img_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_images = train_img_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_images = test_img_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

KeyError: 'Filepath'

In [14]:
print(type(train_images))

NameError: name 'train_images' is not defined

# Model Training

# Model Selection and Hyperparameter Tuning

# Insights and conclusions